In [1]:
#0.導入套件(以前程式抓來的，可能有些用不上)
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.autograd import grad
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler      #torch的太麻煩了，用sklearn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:
#1.讀取檔案，設定參數
#----------------------------------------
data = pd.read_csv("data.csv")
#data = data.rename(columns={"x'": "xp"})     原本考量x'往後需要變成x\'，但發現後面只出現一次，而且用不同之引號能解決'的問題，故注解掉
X = data[['a','m','V0','n',"x'"]].values             
#在 PINN ，psi_raw 不是主 label，僅是輔助監督學習之資料
psi_data = data['psi_raw'].values.reshape(-1, 1)

#2.Normalization(標準化)
#---------------------------------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_tensor = torch.tensor(
    X_scaled,
    dtype=torch.float32,
    requires_grad=True  #確保x要能微分
).to(device)
psi_data = torch.tensor(
    psi_data,
    dtype=torch.float32
).to(device)
E = torch.nn.Parameter(torch.tensor(1.0, device=device))

In [3]:
#3.模型
#---------------------------------------------------------
class PINN(nn.Module):                    #其實跟ANN一模一樣，命名成PINN
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(5 , 128),
            nn.Tanh(),
            nn.Linear(128 , 64),
            nn.Tanh(),
            nn.Linear(64 , 32),
            nn.Tanh(),
            nn.Linear(32 , 1),
        )
    def forward(self , X):
        return self.net(X)

model = PINN().to(device)

optimizer = torch.optim.Adam(list(model.parameters()) + [E], lr=0.001)#因為他自己會找到ψ = 0這條最簡單的路線，強制補E使得必定要存在能量

In [4]:
#3.1 定義PINN專屬參數pDE殘差
#---------------------------------------------------------
def pde_residual(model, X):
    psi = model(X)

    # 一階導數 dψ/dx
    psi_x = grad(
        psi, X,
        grad_outputs=torch.ones_like(psi),
        create_graph=True
    )[0][:, 4:5]   # 只取對 x 的偏導

    # 二階導數 d²ψ/dx²
    psi_xx = grad(
        psi_x, X,
        grad_outputs=torch.ones_like(psi_x),
        create_graph=True
    )[0][:, 4:5]
    
    # 非齊次 source term
    x_phys = X[:, 4:5]
    f = torch.exp(-x_phys**2)
    epsilon0 = 1.0

    residual = psi_xx + E * psi - epsilon * f
    return residual

In [5]:
# 3.2定義loss
#---------------------------------------------------
def pinn_loss(model, X, psi_data=None, lambda_data=0.1, lambda_norm=10):
     # PDE loss
    R = pde_residual(model, X)
    loss_pde = torch.mean(R**2)
    # 標準化 loss（破 ψ≡0）
    psi = model(X)
    norm = torch.mean(psi**2)
    loss_norm = (norm - 1.0)**2
    #原先data之標準化不能用在這，因為它屬於固定解，會限制模型
    x0_index = X.shape[0] // 2      
    psi_anchor = psi[x0_index]
    loss_anchor = (psi_anchor - 1.0)**2

    # data loss（可選）
    if psi_data is not None:
        loss_data = torch.mean((psi - psi_data)**2)
    else:
        loss_data = 0.0

    loss = loss_pde+ lambda_norm * loss_norm+ lambda_data * loss_data+ 0.01 * loss_anchor
    return loss

In [7]:
# 4.訓練
#---------------------------------------------------
epochs = 200
for i in range(epochs):
    optimizer.zero_grad()
    
    loss = pinn_loss(model, X_tensor, psi_data)
    loss.backward()
    optimizer.step()
    #if(i == 0 or (i + 1) % 100 == 0):
    print(f"Epoch{i}, loss={loss.item():.5f}, E={E.iten():.6f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 232.00 MiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 5.21 GiB is allocated by PyTorch, and 78.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
#5.儲存
#--------------------------------------------------
torch.save(model.state_dict(), "pinn_model.pth")

# 儲存 scaler
import joblib
joblib.dump(scaler, "pinn_scaler.pkl")

print("Model and scaler saved.")

Model and scaler saved.
